# Reports analysis

Understand what should be taken from the reports.

In [1]:
# Test with some samples
from IPython.display import display
from lxml import etree, html
import lib.report_parsers as jcfg_report_parsers
import gzip
import json

data_dir = 'data/analyses_gz/'
samples = ['MjhkYjM1ZjY1YWNkNDQ1Y2FiYmM1M2MwNTI0OGEzYjA',
           'MTRiZjc0YTA2MmYwNDA2NDk4MDA1YzU2NzJkY2ZkYjc',
           'MzIwZDgzMjY0YjQ5NGQxMjhkZjk1YjE0YTlkNGQ1OTE',
           'MzMxZjlkMDljZDA3NDViMThmNzMwOWYwYWNhMGY1MmY',
           'YjYxNGFhZTNlZDFkNGI5Yjk1NmEzM2ZlM2EwZWU0YmQ',
           'MzMxZjlkMDljZDA3NDViMThmNzMwOWYwYWNhMGY1MmY',
           'YTU5ZmNiNzM0OTgyNDQxOTgyMGU5N2MwMGFiYjQzMmI',
           'MzYwNjVjZjA4ODgzNGUyOGJiMDMzZWRjNTFlNTcyMDg',
           'Mzg3NTIyMTAzZWVmNDQ3OTgzZTM3MDE5NGJlNzQwNzA',
           'MzExMzhlYzIyMjZlNGI5ZGE1M2E5MTU3NzdkZTNmOTk',
           'NWExYTBjZGVjMzQzNDBlYzg2MjBkM2NjODlhYTcwZWI',
           'Zjk1YzcxNDUzNGFiNDliZWFiOWM3NTE3NDE2YThlM2I',
           'MTkwZTNmNDRlNWU4NDVlZDllNzY1NWQ0NWE1NjQ3OTk',
           'M2Q0OWI2ODJmMTA3NDU5Y2E2MDM3OWNmODlkOWE3OTI',
           'OTY0ODgzZGQyNGVmNGEwNTgxMjlkYTA5MzcwMjBlYjI',
           'ODg3MzI1MzVkMDBmNGMyNTk0ZTgxM2I1MzMzODMxNGQ',
           'NjEwYzNmZDRmYTBiNGQ2NDhmOTcyMjYzY2I3NDdhZWM',
           'YjY2YWMxYTdkN2IwNDc5N2E4MDc4YzFlMjM5YzRlNzE',
           'NWQ4ZDcyYzNjM2M5NDliYzkwOWJmOWQ0MmJjYThlNTU',]

# samples = ['NWExYTBjZGVjMzQzNDBlYzg2MjBkM2NjODlhYTcwZWI']
# samples = ['NWQ4ZDcyYzNjM2M5NDliYzkwOWJmOWQ0MmJjYThlNTU']
# samples = ['ODIyNjI1MjdiOTFhNDY1ZjljZDBhNmE0ODIyMTc0YzY']
# samples = ['OTNkNGFkMTAyYjg1NDc4MWExNjY2MzQ5OTI3MjVjM2I']
# samples = ['N2NlNzIwM2Q3YjYyNDA0MzkxNjBhYWQ1OGI1ZWEzNzU']
# samples = ['MjhkYjM1ZjY1YWNkNDQ1Y2FiYmM1M2MwNTI0OGEzYjA']
# Keys
# samples = ['YWRlNzgyMzk2ZDAwNDQzMmE3Y2ZjM2VhYzQ5OWFkZjU']

def parse_report(sample, location):
    """
        Takes a sample file, loads it and parses the report
    """
    SIZE_LIMIT = 10000000
    location = location + '/' if location[-1] != '/' else location
    func_list = list(filter(lambda x: x.startswith('extract_'), dir(jcfg_report_parsers)))
    
    
    with gzip.open(location + sample) as gz_file:
        content = gz_file.read().decode('utf8')
    
    content = jcfg_report_parsers.remove_whitespaces(content)
    func_cutpoints = jcfg_report_parsers.generate_func_cutpoints(content)
    # Check if any section is over 10MB, otherwise
    # lxml can take it anyway
    report = dict()
    for idx, pair in enumerate(func_cutpoints[1:]):
        if pair[1] > SIZE_LIMIT:
            # Section to parse is the one before the one being compared
            func_cutpoints[idx] = (func_cutpoints[idx][0] + '_huge', func_cutpoints[idx][1])
        # if func_cutpoints[idx][0] == 'extract_mutexes':
            # func_cutpoints[idx] = ('extract_mutexes_huge', func_cutpoints[idx][1])

    # Returns either str if manual parsing or etree 
    input_type = lambda name, content: content if '_huge' in name else etree.HTML(content)
    for func, cutpoint in func_cutpoints:
        content = content[cutpoint:]
        # Turn the string into a function and call it with correct input
        result = getattr(jcfg_report_parsers, func)(input_type(func, content))
        # When saving we don't need the _huge call
        report[func.replace('_huge', '')] = result

    return (sample, report)


import json 
from os import walk

#(_, _, reports) = next(walk(data_dir))
# for sample in reports:
#    parse_report(sample, data_dir)


In [7]:
import tasks
from celery import group

samples = ['NWExYTBjZGVjMzQzNDBlYzg2MjBkM2NjODlhYTcwZWI',]

#reports = []
#for sample in samples:
#    print(sample)
    # Load bunch of gz's into memory
#    reports.append(sample)

# Parse them
# print(len(reports))
# jobs = group([tasks.extract_from_report.s(func_list, rep) for rep in reports])
# result = jobs.apply_async()
# result.join()
                      
# display(len(result.get()))

# Read around 10GB to memory
CONTENT_SIZE = 1000000 * 1000 * 0.01
content_size = 0
content_batch = []
data_dir = 'data/analyses_gz/'
output = 'data/extracted/'
for sample in samples:
    with gzip.open(data_dir + sample) as gz_file:
        content = jcfg_report_parsers.remove_whitespaces(gz_file.read().decode('utf8'))
        content_size += len(content)
        content_batch.append((sample, content))
        display(content_size)
    if content_size >= CONTENT_SIZE:
        # Fire away
        jobs = group([tasks.extract_from_report.s(link, content) for link, content in content_batch])            
        # Wait for workers to finish
        result = jobs.apply_async()
        result.join()
        for r in result.get():
            with open(output + r[0], 'w') as file:
                file.write(json.dumps(r[1]))
        content_size = 0
        content_batch = []
    
# Fire away
jobs = group([tasks.extract_from_report.s(link, content) for link, content in content_batch])            
# Wait for workers to finish
result = jobs.apply_async()
result.join()
for r in result.get():
    with open(output + r[0], 'w') as file:
        file.write(json.dumps(r[1]))
print(result.get())

12847684